In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


from yellowbrick.model_selection import ValidationCurve
from yellowbrick.classifier import ConfusionMatrix

In [ ]:
df = pd.read_csv('Data_for_Modeling.csv')

In [ ]:
df = df.drop(['Unnamed: 0','Unnamed: 0.1','crash_record_id','crash_date','injuries_total','street_no','street_name','location'], axis=1)

In [ ]:
df_cat = df[['traffic_control_device','device_condition','weather_condition','lighting_condition','first_crash_type','trafficway_type','alignment',\
            'roadway_surface_cond','road_defect','crash_type','damage','sec_contributory_cause','most_severe_injury']]
df_num = df[['posted_speed_limit','beat_of_occurrence','injuries_fatal','injuries_incapacitating','injuries_non_incapacitating',\
             'injuries_reported_not_evident','injuries_no_indication','crash_hour','crash_day_of_week','crash_month','num_passengers','latitude','longitude',\
             'num_passengers','vehicle_defect','sex','age','physical_condition']]

In [ ]:
df_dummies = pd.get_dummies(df_cat)
ss = StandardScaler()
df_num_scaled = pd.DataFrame(ss.fit_transform(df_num), columns = ['posted_speed_limit','beat_of_occurrence','injuries_fatal','injuries_incapacitating',\
                                                                  'injuries_non_incapacitating','injuries_reported_not_evident','injuries_no_indication',\
                                                                  'crash_hour','crash_day_of_week','crash_month','num_passengers','latitude','longitude',\
                                                                  'num_passengers','vehicle_defect','sex','age','physical_condition'])
df_final = pd.concat([df_num_scaled,df_dummies,df['prim_contributory_cause']],axis=1)
df_final = df_final.dropna()
df_final.shape
df_sample = df_final.sample(n=10000)

In [ ]:
X = df_sample.drop(columns=['prim_contributory_cause'])
y = df_sample['prim_contributory_cause']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
#Fitting a Dummy Model:
dummy = DummyClassifier(strategy='most_frequent')
dummy.fit(X_train, y_train)
dummy.score(X_test, y_test)

In [ ]:
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train, )
logreg.score(X_train, y_train)

In [ ]:
cm = ConfusionMatrix(logreg)

cm.score(X_train, y_train)

cm.poof()

# Instantiate the classification model and visualizer
y_pred=logreg.predict(X_train)
print(classification_report(y_train, y_pred))

In [ ]:
from sklearn.model_selection import cross_val_score
cv = cross_val_score(logreg, X_train, y_train)
cv

In [ ]:
odds = pd.DataFrame(np.exp(logreg.coef_), columns=X.columns)
odds = odds.T.rename(columns={0:'Coefficient Odds'})

In [ ]:
odds = odds.sort_values(by='Coefficient Odds', ascending=False)

In [ ]:
odds.head(10)

In [ ]:
df.first_crash_type.value_counts()